In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
df = pd.read_excel("Interview_questions.xlsx",sheet_name='base_data')

In [5]:
df.head()

,date,order_no,format,division_name,orderval,rcvval
0,2020-10-18,1,sm,Staples,9732.80,9732.80
1,2020-10-16,2,sm,Grocery Food,558.71,558.71
2,2020-10-20,3,sm,Grocery Non Food,2927.04,2927.04
3,2020-10-21,4,sm,Grocery Food,905.96,905.96
4,2020-10-27,5,sm,Grocery Food,988.62,988.62


#### Filling null values

In [6]:
df['orderval'].fillna(0, inplace = True)
df['rcvval'].fillna(0, inplace = True)


In [7]:
df.isnull().sum()

date             0
order_no         0
format           0
division_name    0
orderval         0
rcvval           0
dtype: int64

In [8]:
df.head(5)

,date,order_no,format,division_name,orderval,rcvval
0,2020-10-18,1,sm,Staples,9732.80,9732.80
1,2020-10-16,2,sm,Grocery Food,558.71,558.71
2,2020-10-20,3,sm,Grocery Non Food,2927.04,2927.04
3,2020-10-21,4,sm,Grocery Food,905.96,905.96
4,2020-10-27,5,sm,Grocery Food,988.62,988.62


In [9]:
df2 = df.copy()

### create a column day_bucket 1

In [10]:
df2['date'] - pd.datetime(2020,10,22)

0       -4 days
1       -6 days
2       -2 days
3       -1 days
4        5 days
          ...  
11901    4 days
11902    1 days
11903    4 days
11904    0 days
11905    6 days
Name: date, Length: 11906, dtype: timedelta64[ns]

In [11]:
#converting from date time object to string
(df2['date'] - pd.datetime(2020,10,22)).astype('str').str[:2]

0        -4
1        -6
2        -2
3        -1
4        5 
         ..
11901    4 
11902    1 
11903    4 
11904    0 
11905    6 
Name: date, Length: 11906, dtype: object

In [13]:
df2['day_bucket 1'] = (df2['date'] - pd.datetime(2020,10,22)).astype('str').str[:2]

In [15]:
df2[['date','day_bucket 1']]

,date,day_bucket 1
0,2020-10-18,-4
1,2020-10-16,-6
2,2020-10-20,-2
3,2020-10-21,-1
4,2020-10-27,5
...,...,...
11901,2020-10-26,4
11902,2020-10-23,1
11903,2020-10-26,4
11904,2020-10-22,0


In [17]:
df.drop(columns = 'date', inplace = True)

In [22]:
df2['day_bucket 2'] = df2['day_bucket 1'].map({
    '-9' : '-1 to -10',
    '-8' : '-1 to -10',
    '-7' : '-1 to -10',
    '-6' : '-1 to -10',
    '-5' : '-1 to -10',
    '-4' : '-1 to -10',
    '-3' : '-1 to -10',
    '-2' : '-1 to -10',
    '-1' : '-1 to -10',
    
        '0 ' : '0 to 2',
        '1 ' : '0 to 2',
        '2 ' : '0 to 2',
            
                '3 ' : '3 to 7',
                '4 ' : '3 to 7',
                '5 ' : '3 to 7',
                '6 ' : '3 to 7',
                '7 ' : '3 to 7'
        
})

In [23]:
df2[['day_bucket 1', 'day_bucket 2']]

,day_bucket 1,day_bucket 2
0,-4,-1 to -10
1,-6,-1 to -10
2,-2,-1 to -10
3,-1,-1 to -10
4,5,3 to 7
...,...,...
11901,4,3 to 7
11902,1,0 to 2
11903,4,3 to 7
11904,0,0 to 2


In [25]:
d1 = df2[df2['day_bucket 2']=='-1 to -10'].groupby(['format','division_name'])[['orderval','rcvval']].sum()
d2 = df2[df2['day_bucket 2']=='0 to 2'].groupby(['format','division_name'])[['orderval','rcvval']].sum()
d3 = df2[df2['day_bucket 2']=='3 to 7'].groupby(['format','division_name'])[['orderval','rcvval']].sum()

In [26]:
d1['fillrate'] = d1['rcvval']/d1['orderval']
d2['fillrate'] = d2['rcvval']/d1['orderval']
d3['fillrate'] = d3['rcvval']/d1['orderval']

In [27]:
d1.coulmns = ['orderval_B1','rcvval_B1','fillrate_B1']
d2.coulmns = ['orderval_B2','rcvval_B2','fillrate_B2']
d3.coulmns = ['orderval_B3','rcvval_B3','fillrate_B3']

C:\Users\Shubham\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.
C:\Users\Shubham\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  
C:\Users\Shubham\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
df_list = [d1,d2,d3]
df_final = pd.concat(df_list,axis=1)
df_final

orderval      rcvval  fillrate    orderval  \
format division_name                                                    
hm     Grocery Food       130172.08   130172.08  1.000000  3553283.40   
       Grocery Non Food    14670.56    14670.56  1.000000  1288835.50   
       Staples                 0.00        0.00       NaN   595349.92   
sm     Grocery Food      6951478.73  6735072.93  0.968869  6604780.07   
       Grocery Non Food  4077619.84  4024782.67  0.987042  4512243.74   
       Staples           5057269.34  4395770.93  0.869199  1394218.96   

                             rcvval  fillrate    orderval     rcvval  fillrate  
format division_name                                                            
hm     Grocery Food        20420.15  0.156870  3050918.15    1899.46  0.014592  
       Grocery Non Food     5112.35  0.348477  2886840.31    3879.31  0.264428  
       Staples                 0.00       NaN  1249676.03       0.00       NaN  
sm     Grocery Food      1720812.39  0.247546  5513566.82  484621.63  0.069715  
       Grocery Non Food  2060888.87  0.505415  4376645.83  303398.31  0.074406  
       Staples             53575.54  0.010594  2045736.20       0.00  0.000000